<a href="https://colab.research.google.com/github/branley1/smol-course/blob/main/1_instruction_tuning/notebooks/sft_finetuning_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the requirements in Google Colab
!pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face
from huggingface_hub import login
!huggingface-cli login

login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [ ]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the SmolLM2 model and tokenizer
model_smol = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-135M").to(device)
tokenizer_smol = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M")

# Set up the chat format for SmolLM2
model_smol, tokenizer_smol = setup_chat_format(model=model_smol, tokenizer=tokenizer_smol)


# Load the GPT-2 tokenizer for ds_code_parrot
tokenizer_gpt2 = AutoTokenizer.from_pretrained("gpt2")
if tokenizer_gpt2.pad_token is None:
    tokenizer_gpt2.pad_token = tokenizer_gpt2.eos_token

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

# Generate with the base model

Here we will try out the base model which does not have a chat template.

In [ ]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer_smol.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer_smol(formatted_prompt, return_tensors="pt").to(device)
outputs = model_smol.generate(**inputs, max_new_tokens=100)
display("Before training:")
display(tokenizer_smol.decode(outputs[0], skip_special_tokens=True))

## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [ ]:
# Load a sample dataset
from datasets import load_dataset, Dataset

# Tokenizer setup
tokenizer = AutoTokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

# Define datasets
# Smoltalk dataset
ds_smoltalk = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

# Process Smoltalk dataset for TRL chat templates
def process_smoltalk(sample):
    # Convert Smoltalk conversation to chat format
    messages = [{"role": msg["role"], "content": msg["content"]} for msg in sample["messages"]]
    return {"messages": messages}

# Apply the processing
ds_smoltalk = ds_smoltalk.map(process_smoltalk, remove_columns=ds_smoltalk["train"].column_names)

# BigCode dataset
ds_big_code = load_dataset(path="bigcode/the-stack-smol-xl", data_dir="data/python")

# Process BigCode dataset for TRL chat templates
def process_big_code(sample):
    # For BigCode, structure the code snippet into a chat format
    messages = [
        {"role": "system", "content": "You are a programming assistant specializing in Python."},
        {"role": "user", "content": "Write or explain the following Python code:"},
        {"role": "assistant", "content": sample["content"]},
    ]
    return {"messages": messages}

# Convert the generator to a list before creating the Dataset
processed_big_code = [process_big_code(sample) for sample in ds_big_code["train"]]
ds_big_code_train = Dataset.from_list(processed_big_code)  # Create a new Dataset object
ds_big_code["train"] = ds_big_code_train  # Replace the original 'train' split with the processed one

# CodeParrot dataset
languages_to_keep = ["Python", "C++", "C", "HTML", "CSS", "SQL", "TypeScript", "TeX"]
ds_code_parrot = load_dataset("codeparrot/github-code", streaming=True, split="train").filter(
    lambda x: x["language"] in languages_to_keep)

# Convert processed CodeParrot datatset into a generator
processed_code_parrot = (process_code_parrot(sample) for sample in ds_code_parrot)

# Process CodeParrot dataset for TRL chat templates
def process_code_parrot(sample):
    messages = [
        {"role": "system", "content": f"You are a programming assistant specializing in {sample['language']}."},
        {"role": "user", "content": "Write or explain the following code snippet:"},
        {"role": "assistant", "content": sample["code"]},
    ]
    return {"messages": messages}

# Map the processing directly on the streaming dataset
processed_code_parrot = ds_code_parrot.map(process_code_parrot)

In [ ]:
# General tokenization function
def tokenize_messages_dataset(dataset,tokenizer):
  """
  Tokenizes a dataset using the specified tokenizer.
  Args:
      dataset: The dataset to tokenize.
      tokenizer: The tokenizer to use.
  Returns:
      Tokenized dataset.
  """
  def tokenize_function(sample):
      chat_str = ""
      for msg in sample.get("messages", []):
        if isinstance(msg, dict): # Check if msg is in a dictionary
          role, content = msg.get("role", "").capitalize(), msg.get("content", "")
          chat_str += f"{role}: {content}\n"

      # Tokenize and pad
      return tokenizer(
          chat_str,
          truncation=True,
          padding="max_length",
          max_length=512,
          return_attention_mask=True,
          )

  return dataset.map(tokenize_function, batched=False, remove_columns=["messages"])

# Tokenization function for datasets with 'code'
def tokenize_code_dataset(dataset, tokenizer):
    """
    Tokenizes datasets that have a 'code' field.
    """
    def tokenize_function(sample):
        code_str = sample.get("code", "")
        return tokenizer(
            code_str,
            truncation=True,
            padding="max_length",
            max_length=512,
            return_attention_mask=True
        )

    return dataset.map(
        tokenize_function,
        batched=False, # Process one sample at a time
        remove_columns=["code", "repo_name", "path", "language", "license", "size"]
    )

def add_labels_to_dataset(dataset):
    """
    Adds a 'labels' field to the dataset, which is a copy of 'input_ids'.
    """
    def add_labels(batch):
      # Ensure 'input_ids' is a list before copying
        if isinstance(batch["input_ids"], list):
          batch["labels"] = batch["input_ids"].copy()
        else:
          raise KeyError("'input_ids' should be a list of integers.")
        return batch

    return dataset.map(add_labels)

# Tokenize Smoltalk and BigCode datasets and save to disk
tokenized_smoltalk = tokenize_messages_dataset(ds_smoltalk, tokenizer_smol)
tokenized_smoltalk.save_to_disk("tokenized_smoltalk")

tokenized_big_code = tokenize_messages_dataset(ds_big_code["train"], tokenizer_smol)
tokenized_big_code.save_to_disk("tokenized_big_code")

# Tokenize CodeParrot dataset
# Since 'ds_code_parrot' is an IterableDataset, take a subset and convert it to a regular Dataset
tokenized_code_parrot_iterable = tokenize_code_dataset(ds_code_parrot, tokenizer_gpt2).take(512)
# Convert the IterableDataset to a list to materialize it
tokenized_code_parrot_list = list(tokenized_code_parrot_iterable)
# Create an in-memory Dataset from the list
tokenized_code_parrot_dataset = Dataset.from_list(tokenized_code_parrot_list)
tokenized_code_parrot_dataset.save_to_disk("tokenized_code_parrot")

# Convert the IterableDataset to a list
tokenized_code_parrot_list = list(tokenized_code_parrot_iterable)

# Create an in-memory Dataset from the list
tokenized_code_parrot_dataset = Dataset.from_list(tokenized_code_parrot_list)

# Add labels to all tokenized datasets
tokenized_smoltalk = add_labels_to_dataset(tokenized_smoltalk)
tokenized_big_code = add_labels_to_dataset(tokenized_big_code)
tokenized_code_parrot = add_labels_to_dataset(tokenized_code_parrot_dataset)

# Inspect the first sample of each tokenized dataset
# print("Smoltalk Sample:", tokenized_smoltalk["train"][0])
# print("BigCode Sample:", tokenized_big_code[0])
# print("CodeParrot Sample:", tokenized_code_parrot_dataset[0])

In [ ]:
# For Smoltalk and BigCode (using SmolLM2)
model_name_smol = "HuggingFaceTB/SmolLM2-135M"
model_smol = AutoModelForCausalLM.from_pretrained(model_name_smol).to(device)
tokenizer_smol = AutoTokenizer.from_pretrained(model_name_smol)
model_smol, tokenizer_smol = setup_chat_format(model=model_smol, tokenizer=tokenizer_smol)

# For CodeParrot (using gpt2)
model_name_gpt2 = "gpt2"  # Or a specific gpt2 variant like "gpt2-medium"
model_gpt2 = AutoModelForCausalLM.from_pretrained(model_name_gpt2).to(device)
tokenizer_gpt2 = AutoTokenizer.from_pretrained(model_name_gpt2)
model_gpt2, tokenizer_gpt2 = setup_chat_format(model=model_gpt2, tokenizer=tokenizer_gpt2)


## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [ ]:
!pip install scikit-learn
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sklearn.model_selection import train_test_split
from datasets import load_from_disk
from torch.utils.data import DataLoader
from accelerate import Accelerator

# 1. Initialize Accelerator for mixed precision and optimized training
accelerator = Accelerator(mixed_precision='fp16')

# 2. Load Pre-trained Model and Tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2').to(accelerator.device)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# 3. Load Pre-tokenized Datasets
tokenized_smoltalk = load_from_disk("tokenized_smoltalk")
tokenized_big_code = load_from_disk("tokenized_big_code")
tokenized_code_parrot = load_from_disk("tokenized_code_parrot").select(range(512))

# Split CodeParrot dataset into training and Evaluation Sets (90% train, 10% eval)
split_dataset = tokenized_code_parrot_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# SFTConfig for SmolLM2
sft_config_smol = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    num_train_epochs=2,
    per_device_train_batch_size=8,  # Increase from 4 to 8
    gradient_accumulation_steps=2, # Accumulates gradients over 2 steps
    learning_rate=5e-5,  # Default LR for fine-tuning
    max_seq_length=256,
    logging_steps=100,  # Reduced logging steps
    save_steps=1000,  # Reduced checkpoint frequency
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=500,  # Evaluate every 100 steps
    fp16=True, # Enable mixed precision
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id="finetuned_smol_smolLM2",
)

# SFTTrainer for Smoltalk (SmolLM2)
trainer_smol_smoltalk = SFTTrainer(
    model=model_smol,
    args=sft_config_smol,
    train_dataset=tokenized_smoltalk["train"],
    tokenizer=tokenizer_smol,
    eval_dataset=tokenized_smoltalk.get("test"),
)

# SFTTrainer for BigCode (SmolLM2)
trainer_smol_bigcode = SFTTrainer(
    model=model_smol,
    args=sft_config_smol,
    train_dataset=tokenized_big_code,
    tokenizer=tokenizer_smol,
    eval_dataset=tokenized_big_code,
)

# Configure the SFTTrainer for GPT-2
sft_config_gpt2 = SFTConfig(
    output_dir="./sft_output_codeparrot",
    max_steps=3000, # Increase for larger datasets
    per_device_train_batch_size=8, # Adjust for GPU memory
    gradient_accumulation_steps=2, # Accumulate gradients over 2 steps
    learning_rate=5e-5,
    max_seq_length=256,
    fp16=True,
    logging_steps=50, # Log every 50 steps
    save_steps=500, # Save checkpoints every 500 steps
    evaluation_strategy="steps", # Evaluate at regular intervals
    eval_steps=100, # Evaluate every 100 steps
    hub_model_id="finetuned_gpt2_codeparrot",
)


# Apply tokenization to the streaming dataset. First, convert it to an in-memory
# dataset using take(N) to fetch N elements, toList(N) converts it to a regular Python list
# Instead of creating a new Dataset object, map the tokenization function directly on the streaming dataset
# and then take a subset of the processed data.
tokenized_code_parrot = list(ds_code_parrot.map(
    lambda x: tokenizer_gpt2(
        x["code"],  # Access the 'code' from the sample
        truncation=True,
        padding="max_length",
        max_length=512,
        return_attention_mask=True
    ),
    batched=True,
    remove_columns=["repo_name", "path", "language", "license", "size"] # Remove all original columns
).take(512)) # Convert to list using list()

tokenized_code_parrot_dataset = Dataset.from_list(tokenized_code_parrot)

# SFTTrainer for CodeParrot (GPT-2)
trainer_gpt2_codeparrot = SFTTrainer(
    model=model_gpt2,  # Using GPT-2 for CodeParrot
    args=sft_config_gpt2,
    train_dataset=train_dataset,
    tokenizer=tokenizer_gpt2,
    eval_dataset=eval_dataset,
)

# 7. Prepare with Accelerator
trainer_smol_smoltalk_acc = accelerator.prepare(trainer_smol_smoltalk)
trainer_smol_bigcode_acc = accelerator.prepare(trainer_smol_bigcode)
trainer_gpt2_codeparrot_acc = accelerator.prepare(trainer_gpt2_codeparrot)

# Initialize DataLoader with Optimized Settings
train_dataloader = DataLoader(
    tokenized_smoltalk,
    batch_size=sft_config_smol.per_device_train_batch_size,
    shuffle=True,
    num_workers=8,            # Based on your CPU cores
    pin_memory=True,          # Speeds up data transfer to GPU
    prefetch_factor=4,        # Based on memory
)

train_dataloader = DataLoader(
    tokenized_big_code,
    batch_size=sft_config_smol.per_device_train_batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    prefetch_factor=2,
)

train_dataloader = DataLoader(
    tokenized_code_parrot,
    batch_size=sft_config_gpt2.per_device_train_batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    prefetch_factor=2,
)

## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [ ]:
import wandb

# 1. Log in to your WandB account
wandb.login(key="4d10e9f814a4d0d285b06e84a8ceed2156ec6ed6")

# wandb.init(project="your_project_name", entity="smol-course")

# Train the models
trainer_smol_smoltalk.train()
trainer_smol_big_code.train()
trainer_gpt2_code_parrot.train()

# Save the models
trainer_smol_smoltalk.save_model(f"./finetuned_smol_smoltalk")
trainer_smol_smoltalk.push_to_hub(
    repo_id="bmmasi1/finetuned_smol_smoltalk",
    tags=["smol-course", "smoltalk", "conversation", "fine-tuned"]
)
trainer_smol_big_code.save_model(f"./finetuned_smol_bigcode")
trainer_smol_bigcode.push_to_hub(
    repo_id="bmmasi1/finetuned_smol_bigcode",
    tags=["smol-course", "bigcode", "code", "fine-tuned"]
)
trainer_gpt2_code_parrot.save_model(f"./finetuned_gpt2_codeparrot")
trainer_gpt2_code_parrot.push_to_hub(
    repo_id="bmmasi1/finetuned_gpt2_codeparrot",
    tags=["smol-course", "codeparrot", "code", "fine-tuned"]
)

In [ ]:
# trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [ ]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.